In [1]:
import xarray as xr
test = xr.open_mfdataset('/lsdf/kit/scc/projects/abcde/1980/*/ERA5.pl.temperature.nc', combine='by_coords')
test_data = test.sel(time=slice('1980-01-14T16:00:00','1980-01-15T07:00:00'),level = slice(978,1000), longitude=slice(6,20.75), latitude=slice(55.5,43.75))
test_data = test_data.transpose('time', 'latitude', 'longitude', 'level')
test_data

<xarray.Dataset>
Dimensions:    (latitude: 48, level: 1, longitude: 60, time: 16)
Coordinates:
  * longitude  (longitude) float32 6.0 6.25 6.5 6.75 ... 20.0 20.25 20.5 20.75
  * latitude   (latitude) float32 55.5 55.25 55.0 54.75 ... 44.25 44.0 43.75
  * level      (level) int32 1000
  * time       (time) datetime64[ns] 1980-01-14T16:00:00 ... 1980-01-15T07:00:00
Data variables:
    t          (time, latitude, longitude, level) float32 dask.array<chunksize=(16, 48, 60, 1), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.6
    history:      2020-12-15 23:51:33 GMT by grib_to_netcdf-2.16.0: /opt/ecmw...

In [2]:
from lossycomp.encodings import *
import numpy as np
lat = test_data['t'].coords['latitude'].values
lon = test_data['t'].coords['longitude'].values

In [6]:
lat_st = np.stack([encode_lat(x) for x in lat])
lon_st = np.stack([encode_lon(x) for x in lon])

In [8]:
lat1, lat2 = np.hsplit(lat_st, 2)
lon1, lon2 = np.hsplit(lon_st, 2)

In [9]:
#xx, yy = np.meshgrid(lon1, lat1)
#xx2, yy2 = np.meshgrid(lon2, lat2)

xx, yy = np.meshgrid(lon, lat)

In [10]:
#coords_lat = np.concatenate([[xx]] * len(test_data.time), axis=0)
#coords_lon = np.concatenate([[yy]] * len(test_data.time), axis=0)
#coords_lat1 = np.concatenate([[xx2]] * len(test_data.time), axis=0)
#coords_lon1 = np.concatenate([[yy2]] * len(test_data.time), axis=0)

coords_lat = np.concatenate([[xx]] * len(test_data.time), axis=0)
coords_lon = np.concatenate([[yy]] * len(test_data.time), axis=0)

In [11]:
#coords_lat = np.expand_dims(coords_lat, axis=3)
#coords_lon = np.expand_dims(coords_lon, axis=3)

#coords_lat1 = np.expand_dims(coords_lat1, axis=3)
#coords_lon1 = np.expand_dims(coords_lon1, axis=3)


coords_lat = np.expand_dims(coords_lat, axis=3)
coords_lon = np.expand_dims(coords_lon, axis=3)

In [12]:
temp = test_data['t'].values

In [13]:
#all_val =  np.concatenate((temp, coords_lat, coords_lon, coords_lat1, coords_lon1 ),axis = 3)
all_val =  np.concatenate((temp, coords_lat, coords_lon),axis = 3)

In [14]:
all_val.shape

(16, 48, 60, 3)

In [15]:
all_val[0,0,0,:]

array([276.35477,   6.     ,  55.5    ], dtype=float32)

In [16]:
test_data.isel(latitude = 3, longitude = 1).t.values

array([[276.54477],
       [276.76416],
       [276.74606],
       [276.70084],
       [276.7958 ],
       [276.89758],
       [276.70987],
       [276.88855],
       [276.979  ],
       [276.98807],
       [276.979  ],
       [276.94736],
       [276.9383 ],
       [276.94736],
       [277.01068],
       [277.11926]], dtype=float32)

In [17]:
all_val[:,3,1,0]

array([276.54477, 276.76416, 276.74606, 276.70084, 276.7958 , 276.89758,
       276.70987, 276.88855, 276.979  , 276.98807, 276.979  , 276.94736,
       276.9383 , 276.94736, 277.01068, 277.11926], dtype=float32)

## Testing

In [1]:
import numpy as np
import dask
from collections import OrderedDict
from lossycomp.dataLoader import DataGenerator, data_preprocessing, split_data

In [6]:
dask.config.set(**{'array.slicing.split_large_chunks': False})

file = '/lsdf/kit/scc/projects/abcde/1979/*/ERA5.pl.temperature.nc'
region = "globe"
var = OrderedDict({'t': 1000})

print("1")
z, mean, std = data_preprocessing(file, var, region)

print("2")
train, test = split_data(z, 0.70)

print("3")
leads = dict(time = 16, longitude=49, latitude=49, level=1)

print("4")
dg_train = DataGenerator(train, 10, leads, batch_size=10, load=True, mean= mean, std=std, coords = True) 

1
2
3
4
Loading data into RAM


In [5]:
dg_train.__getitem__(0)[0][0].shape

(16, 49, 49, 5)

In [ ]:
g